<a href="https://colab.research.google.com/github/aadilganigaie/ChatAnalyzer/blob/main/Filtersize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tensorflow import keras
from tensorflow.keras import layers
#import keras_tuner
#from keras_tuner.tuners import RandomSearch
import nlpaug as naw
import string
from nltk.corpus import stopwords
#from nltk.stem.porter import PortorStemmer
import re
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

stop_words = stopwords.words("english")
import contractions
#import fasttext
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, GlobalMaxPooling1D, Flatten, Conv1D, MaxPool1D, Reshape

from keras.layers.convolutional import Conv1D
from keras.preprocessing.text import Tokenizer
toke = Tokenizer()
import contractions
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential

In [ ]:
df = pd.read_csv('/content/train-balanced-sarcasm.csv', error_bad_lines= False)
df

In [ ]:
df.isna().sum()


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(df['label'])
labels

In [ ]:
def clean_text(text):
    text = text.lower()
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    text =  text.encode('ascii', 'ignore').decode()
    text = re.sub("@\S+", " ", text)
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\'\w+", '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub(r'\w*\d+\w*', '', text)
    text = re.sub('\s{2,}', " ", text)
    return text
  df['clean_text'] = df['text'].apply(clean_text)

In [ ]:
df['no_contrac'] = df['clean_text']. apply(lambda x: [contractions.fix(word) for word in x.split()])
df['no_contrac_str'] = [' '.join(map(str, l)) for l in df['no_contrac']]

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['lem_sent'] = [lemmatizer.lemmatize(words_sent) for words_sent in df['clean_text']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['lem_sent'],df['label'], test_size = 0.2, shuffle= False)

In [ ]:
toke.fit_on_texts(x_train)
vocab_size = len(toke.word_index) +1
vocab_size

In [ ]:
x_train = toke.texts_to_sequences(x_train)
x_test = toke.texts_to_sequences(x_test)

In [ ]:
x_train_pad = pad_sequences(x_train, padding = 'post', maxlen = 200)
x_test_pad = pad_sequences(x_test, padding = 'post', maxlen = 200)

In [ ]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential

# Set the filter sizes you want to try
filter_sizes = [2, 3, 4]

# Set the number of filters you want to try
num_filters_list = [32, 64, 128]

# Create an empty list to store the models
models = []
history = []

# Loop over each filter size
for filter_size in filter_sizes:
    # Loop over each number of filters
    for num_filters in num_filters_list:
        # Define the model
        model = Sequential()
        model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=200))
        model.add(Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu'))
        model.add(GlobalMaxPooling1D())
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model
        history = model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # Add the model to the list
        models.append(model)

# Fit the models
for model in models:
    model.fit(x_train_pad, y_train, validation_data=(x_test_pad, y_test), epochs=10, batch_size=64)

# Evaluate the models
for model in models:
    print(model.evaluate(x_test_pad, y_test))


In [ ]:
import matplotlib.pyplot as plt

# Loop over the models
for i, model in enumerate(models):
    # Get the training and validation loss and accuracy for each epoch
    loss = model.history.history['loss']
    val_loss = model.history.history['val_loss']
    acc = model.history.history['accuracy']
    val_acc = model.history.history['val_accuracy']

    # Plot the training and validation loss
    plt.figure(figsize=(10, 5))
    plt.plot(loss, label='Training loss')
    plt.plot(val_loss, label='Validation loss')
    plt.legend(loc='best')
    plt.title('Model {} Loss'.format(i+1))
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

    # Plot the training and validation accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(acc, label='Training accuracy')
    plt.plot(val_acc, label='Validation accuracy')
    plt.legend(loc='best')
    plt.title('Model {} Accuracy'.format(i+1))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.show()


In [ ]:
import seaborn as sns
y_pred = models[0].predict(x_test_pad)

# Convert the continuous-multioutput predictions to one-hot encoded predictions
y_pred = y_pred.argmax(axis=1)


# Get the confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot = True)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

# Loop over the models
for i, model in enumerate(models):
    # Get the predictions for the test set
    y_pred = model.predict(x_test_pad)

    # Convert the continuous-multioutput predictions to one-hot encoded predictions
    y_pred = y_pred.argmax(axis=1)

    # Get the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 3))

    sns.heatmap(cm/np.sum(cm), annot=True,
            fmt='.2%')



